GBIF processing follows the same pattern as all of the other SppIn information gatherers. Running locally, I pull all messages and then process them in parallel at a rate that should not break anything. When running in a lambda environment, we will need to similarly throttle the number of concurrent connections we send to the GBIF API.

In [1]:
import pysgcn
sgcn = pysgcn.sgcn.Sgcn()

from joblib import Parallel, delayed
from tqdm import tqdm

mq = "mq_gbif_check"
sppin_source = "gbif"

In [2]:
messages = sgcn.sql_mq.get_all_records("mq", mq)
print(len(messages))
messages[99]

40194


{'id': 'a32f53b70969891865c15889268f1d4bb9804053',
 'date_inserted': '2019-12-19T13:22:10.094838',
 'body': {'source': {'type': 'List of Scientific Names',
   'name_source': 'ITIS Search'},
  'sppin_key': 'Scientific Name:Polyodon spathula'}}

In [ ]:
%%time
Parallel(n_jobs=8, prefer="threads")(
    delayed(sgcn.process_sppin_source_search_term)
    (
        message_queue=mq,
        sppin_source=sppin_source,
        message_id=message["id"], 
        message_body=message["body"]
    ) for message in tqdm(messages)
)